# Introduction

This notebook is the computational appendix of arXiv:YET-TO-KNOW. We demonstrate how to use a modified version of the NPA hierarchy(https://arxiv.org/abs/0903.4368) to test whether a set of observed correlations is compatible with a separable state. We also give the details how to reproduce the numerical results shown in the paper. Furthermore, we show how to determine the robustness to noise of a set of given observed correlations and to extract an entanglement witness from the dual of the SDP. To improve readability of this notebook, we placed the supporting functions to a separate file; please download that in the same folder as the notebook if you would like to evaluate it. The following dependencies must also be available: at least one SDP solver ([SDPA](http://sdpa.sourceforge.net) as an executable in the path or [Mosek](https://mosek.com) with its Python interface installed; cvxopt as a solver is not recommended) together with the [Ncpol2sdpa](http://pypi.python.org/pypi/ncpol2sdpa) package.
First, we import everything we will need:

In [3]:
from sympy import S
from ncpol2sdpa import flatten, SdpRelaxation, generate_variables
from time import time
from entanglement_tools import *
from math import pi

We illustrate the algorithm for the quench experiment described in Section IV.A in the paper (a single spin flip in an $XX$ chain, starting from a ferromagnetic state). We define the relevant parameters of the scenario: the number of measurement per qubit $m = 3$ (since we use correlation related to the three Pauli measurements), the number of qubits $N$ and an optional parameter to define which correlations are observed. We also fix the time $t$ after the initial quench, for which compute the corresponding correlations (for further details on the quench experiment, see the paper). 

In [9]:
m = 3 #3 spin measurements per party, along direction x, y, z
N = 20 #number of parties / qubits
t = 0.1 #time after the quench to generate the data
full_info = False #if False, generates the data <Zi>, <Zi Zj> and <Xi Xj>=<Yi Yj>. If True, generates also <Xi Yj>.

## Definition of the SDP problem and extraction of the noise robustness for the 1D quench experiment

We now define the symbolic variables associated to the three measurements per qubit, together with the parameter $\lambda$ representing the white noise added to the observed correlations.

In [10]:
d = 2 #for qubits. qudits (d>2) have not been implemented in this code.
configuration = [d for _ in range(m)] #we will generate d-1(=1) symbolic variables, for m measurement operators. In general (d>2), this could be used to also define higher powers of the local measurements.

measurements = [generate_commuting_measurements(configuration, chr(65+i)) for i in range(N)] #generates the symbolic variables, for N parties

print("The Pauli measurements X,Y,Z are denoted as [Z0,X0,Y0]=[A0,A1,A2]; [Z1,X1,Y1]=[B0,B1,B2]...")
for i in range(N):
    print("qubit {}: measurements {}".format(i, measurements[i]))

lambda_ = generate_variables("\lambda")[0] #symbolic variable representing the noise

The Pauli measurements X,Y,Z are denoted as [Z0,X0,Y0]=[A0,A1,A2]; [Z1,X1,Y1]=[B0,B1,B2]...
qubit 0: measurements [[A0], [A1], [A2]]
qubit 1: measurements [[B0], [B1], [B2]]
qubit 2: measurements [[C0], [C1], [C2]]
qubit 3: measurements [[D0], [D1], [D2]]
qubit 4: measurements [[E0], [E1], [E2]]
qubit 5: measurements [[F0], [F1], [F2]]
qubit 6: measurements [[G0], [G1], [G2]]
qubit 7: measurements [[H0], [H1], [H2]]
qubit 8: measurements [[I0], [I1], [I2]]
qubit 9: measurements [[J0], [J1], [J2]]
qubit 10: measurements [[K0], [K1], [K2]]
qubit 11: measurements [[L0], [L1], [L2]]
qubit 12: measurements [[M0], [M1], [M2]]
qubit 13: measurements [[N0], [N1], [N2]]
qubit 14: measurements [[O0], [O1], [O2]]
qubit 15: measurements [[P0], [P1], [P2]]
qubit 16: measurements [[Q0], [Q1], [Q2]]
qubit 17: measurements [[R0], [R1], [R2]]
qubit 18: measurements [[S0], [S1], [S2]]
qubit 19: measurements [[T0], [T1], [T2]]


We introduce the Pauli constraints for the variables, namely $x_i^2 + y_i^2 + z_i^2 = 1$ for all qubits $i = 1,\ldots,N$.

In [11]:
pauli_eqs = []

for party in measurements:
    pauli_eqs.append(S.One - party[0][0]**2 - party[1][0]**2 - party[2][0]**2) #symbolic representation of the constraint

Lastly, we compute the correlations $C_r$ resulting from the 1D quench experiment (see the paper) and create a dictionary to associate the symbolic expression $C_r(1-\lambda)$ to the corresponding variable ($A_0$, $A_0 * B_0$, ...)

In [12]:
time0 = time()    
    
moments = get_moment_quench(N, t, measurements, lambda_, full_info)

print("Moments were generated in " + str(time()-time0) + " seconds.")
print("The moments are (1-\lambda) times the correlator:")
moments 

Moments were generated in 0.35382652282714844 seconds.
The moments are (1-\lambda) times the correlator:


{A0: 0.990018732648381*\lambda - 0.990018732648381,
 A1: 0,
 A2: 0,
 B0: 0.995012486986759 - 0.995012486986759*\lambda,
 B1: 0,
 B2: 0,
 C0: 0.999996880204537 - 0.999996880204537*\lambda,
 C1: 0,
 C2: 0,
 D0: 0.999999999133029 - 0.999999999133029*\lambda,
 D1: 0,
 D2: 0,
 E0: 0.999999999999865 - 0.999999999999865*\lambda,
 E1: 0,
 E2: 0,
 F0: 1.0 - 1.0*\lambda,
 F1: 0,
 F2: 0,
 G0: 1.0 - 1.0*\lambda,
 G1: 0,
 G2: 0,
 H0: 1.0 - 1.0*\lambda,
 H1: 0,
 H2: 0,
 I0: 1.0 - 1.0*\lambda,
 I1: 0,
 I2: 0,
 J0: 1.0 - 1.0*\lambda,
 J1: 0,
 J2: 0,
 K0: 1.0 - 1.0*\lambda,
 K1: 0,
 K2: 0,
 L0: 1.0 - 1.0*\lambda,
 L1: 0,
 L2: 0,
 M0: 1.0 - 1.0*\lambda,
 M1: 0,
 M2: 0,
 N0: 1.0 - 1.0*\lambda,
 N1: 0,
 N2: 0,
 O0: 1.0 - 1.0*\lambda,
 O1: 0,
 O2: 0,
 P0: 1.0 - 1.0*\lambda,
 P1: 0,
 P2: 0,
 Q0: 0.999999999999865 - 0.999999999999865*\lambda,
 Q1: 0,
 Q2: 0,
 R0: 0.999999999133029 - 0.999999999133029*\lambda,
 R1: 0,
 R2: 0,
 S0: 0.999996880204537 - 0.999996880204537*\lambda,
 S1: 0,
 S2: 0,
 T0: 0.995012486

We are now ready to define the SDP, corresponding to the hiearchy defined in Appendix A.3 in the article at level $k$. Notice that "momentssubstitutions" take care of enforcing the values of the observed correlators, while the equalities take care of enforcing the Pauli constraints (see Appendix A.3 in the paper for further details). 

In [13]:
time0 = time()
k = 1
    
sdp = SdpRelaxation(flatten([S.One,measurements]), parameters=[lambda_],verbose=1)
sdp.get_relaxation(k, objective=lambda_, #we minimize the noise
                   momentsubstitutions=moments, #compatibility with the available correlations
                   equalities=pauli_eqs #ensures the constraint x_i^2 + y_i^2 + z_i^2 = 1
                )
print("SDP relaxation was generated in " + str(time()-time0) + " seconds.")
sdp.write_to_file("ouput_sdp.csv") #SDP written in a matrix form, save in the correspondin file.

The problem has 61 commuting variables, and 1symbolic parameters
Calculating block structure...
Estimated number of SDP variables: 1891
Generating moment matrix...
Reduced number of SDP variables: 61 61 (done: 100.00%, ETA 00:00:0.0)
Processing 40/40 constraints...
SDP relaxation was generated in 4.220065116882324 seconds.


Solving the SDP yields the maximal noise robustness $\lambda_{max}$ for the entanglement of the observed correlations 

In [14]:
time0 = time()
    
sdp.solve(solver="sdpa")

print("SDP relaxation was solved in " + str(time()-time0) + " seconds.")
print("lambda_max is equal to " + str(sdp.primal))
print("moment matrix for the noise at which it enters the relaxation:")
print(sdp.x_mat[1])

SDP relaxation was solved in 0.29314327239990234 seconds.
lambda_max is equal to 0.00859969275450807
moment matrix for the noise at which it enters the relaxation:
[[ 1.000e+00 -9.815e-01  0.000e+00 ...  9.865e-01  0.000e+00  0.000e+00]
 [-9.815e-01  9.943e-01  0.000e+00 ... -9.864e-01  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  2.867e-03 ...  0.000e+00  2.097e-15  0.000e+00]
 ...
 [ 9.865e-01 -9.864e-01  0.000e+00 ...  9.901e-01  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  2.097e-15 ...  0.000e+00  4.945e-03  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00  4.945e-03]]


## Extraction the entanglement witness

The direct substitution of the correlations values in some entries of the SDP matrix, imposed by the \textit{momentssubstitutions} command, although convenient numerically, erased the information of which entry of the matrix corresponds to which correlation. For this reason, it is convienent to re-define a second SDP sharing the same dual matrix as the previously solved one, so to be able to retrieve the entries corresponding to the witness coefficients.  

In [15]:
time0 = time()

sdp2 = SdpRelaxation(flatten([S.One, measurements]), verbose=0)
sdp2.get_relaxation(k, equalities=pauli_eqs)

sdp2.status = sdp.status
sdp2.y_mat = [sdp.y_mat[1]] #dual variables corresponding to the data (correlations)

print("Second SDP relaxation was generated in " + str(time()-time0) + " seconds.")

Second SDP relaxation was generated in 0.36659717559814453 seconds.


Now we can extract directly the $w_i^a$ and the $w_{ij}^{ab}$ coefficients of the witness from the dual of the second SDP. Notice that the violation obtained by the correlations coincides exactly with the maximal noise $\lambda_{max}$, as expected

In [16]:
ineq = []
for monomial, value in moments.items():
    ineq.append(sdp2.extract_dual_value(monomial) * monomial)
print("The witness takes the form (recall the notation: for qubit 1, [A0,A1,A2]=[Z1,X1,Y1], for qubit 2 [B0,B1,B2]=[Z2,X2,Y2]...):")
print("<W>_{sep} = " + str(sum(ineq)))
print("As explained in the paper, evaluated on the data, it takes the value <W>=1, while for separable states it obeys <W>_{sep} < " + str(1-sdp.primal))

The witness takes the form (recall the notation: for qubit 1, [A0,A1,A2]=[Z1,X1,Y1], for qubit 2 [B0,B1,B2]=[Z2,X2,Y2]...):
<W>_{sep} = -0.2642*A0*B0 + 0.1341*A0*C0 + 0.001808*A0*D0 + 0.001812*A0*E0 + 0.001809*A0*F0 + 0.001809*A0*G0 + 0.001809*A0*H0 + 0.001809*A0*I0 + 0.001809*A0*J0 + 0.001809*A0*K0 + 0.001809*A0*L0 + 0.001809*A0*M0 + 0.001809*A0*N0 + 0.001809*A0*O0 + 0.001809*A0*P0 + 0.001812*A0*Q0 + 0.001808*A0*R0 + 0.1341*A0*S0 - 0.2642*A0*T0 + 0.001686*A0 - 2.132e-8*A1*B1 - 0.1341*A1*C1 - 1.19e-13*A1*D1 + 0.0002227*A1*E1 - 4.251e-13*A1*F1 - 1.864e-8*A1*G1 - 3.815e-14*A1*H1 + 1.221e-12*A1*I1 - 1.431e-14*A1*J1 + 1.908e-14*A1*K1 + 2.146e-13*A1*L1 + 8.012e-13*A1*M1 + 6.009e-13*A1*N1 - 1.864e-8*A1*O1 + 1.407e-13*A1*P1 + 0.0002227*A1*Q1 + 3.054e-13*A1*R1 - 0.1341*A1*S1 + 2.132e-8*A1*T1 - 2.132e-8*A2*B2 - 0.1341*A2*C2 - 1.19e-13*A2*D2 + 0.0002227*A2*E2 - 4.251e-13*A2*F2 - 1.864e-8*A2*G2 - 3.815e-14*A2*H2 + 1.221e-12*A2*I2 - 1.431e-14*A2*J2 + 1.908e-14*A2*K2 + 2.146e-13*A2*L2 + 8.012e-13*A